This file, once complete, will obtain a dataframe that contains the cost per point table from CF for the 2017-18 NHL season. I am going to scrape the CapFriendly Website using beautiful soup and selenium and then use pandas to get it into the dataframe.

In [1]:
import requests
import pandas as pd
import time
import logging
import lxml
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

Now that necessary packages are installed, going to parse the needed url. This URL has a dynamic table that contains the cost per Point of NHL Players for the 2017-18 season. So I am going to scrape the cap friendly website (Cost Per Point tab) and get this into a dataframe that I can eventually download and put into the MySQL database. As mentioned, since the table is dynamic and not static, the selenium package is needed because the table only expands to its full length once you scroll down.

In [2]:
# Set up the Selenium driver (make sure you have the correct path to your WebDriver)
driver_path = 'C:/STL Blues Analytics/SalaryCap/NHLSalaryCap/chromedriver.exe'  # Update this to your WebDriver path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Open the webpage
url = "https://www.capfriendly.com/cost-per-point/2018"
driver.get(url)

# Get initial page height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(5)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    last_height = new_height

# Get the page source after all rows are loaded
page_source = driver.page_source
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find the table and convert it to a DataFrame
table = soup.find('table')
dfattempt = pd.read_html(str(table))[0]



C:\Users\nsofi\AppData\Local\Temp\ipykernel_26528\419894306.py:36: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfattempt = pd.read_html(str(table))[0]


In [3]:
dfattempt.shape
dfattempt.tail(5)

,#,PLAYER,POS,AGE,TYPE,EXPIRY,CAP HIT,P. BONUSES,GP,SH,G,A,P,TOI,$/SH,$/G,$/A,$/P,$/MIN
802,803,"Criscuolo, Kyle",C,25,NaN,Stnd,UFA,"$650,000",9,5,0,0,0,08:00,"$130,000",∞,∞,∞,"$9,027"
803,804,"Beleskey, Matt",LW,29,NaN,Stnd,UFA,"$3,800,000",15,17,0,0,0,10:58,"$223,529",∞,∞,∞,"$23,100"
804,805,"Hanley, Joel",LD/RD,26,NaN,Stnd,UFA,"$725,000",5,2,0,0,0,11:07,"$362,500",∞,∞,∞,"$13,043"
805,806,"Mermis, Dakota",LD/RD,23,NaN,ELC,RFA,"$626,667",9,7,0,0,0,10:59,"$89,523",∞,∞,∞,"$6,339"
806,807,"Holzer, Korbinian",RD,29,NaN,Stnd,UFA,"$900,000",16,10,0,0,0,11:36,"$90,000",∞,∞,∞,"$4,849"


In [4]:
dfattempt.to_csv("CF Cost Per Point 2017-18.csv") 